In [0]:
from google.colab import drive
drive.mount('gdrive/')

Drive already mounted at gdrive/; to attempt to forcibly remount, call drive.mount("gdrive/", force_remount=True).


In [0]:
cd gdrive/'My Drive'/

/content/gdrive/My Drive


In [0]:
import pandas as pd
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
labels=train['category']
from tensorflow.keras.utils import to_categorical
labels_one_hot = to_categorical(labels)
labels_one_hot=labels_one_hot[:,1:]

In [0]:
d={1:'Cargo', 
2:'Military', 
3:'Carrier', 
4:'Cruise', 
5:'Tankers'}
train['category'] .replace(d, inplace=True)

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [18]:
import pandas as pd
df=train
datagen=keras.preprocessing.image.train_datagen = ImageDataGenerator(rescale = None,
                                   rotation_range = 30,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True)
train_generator=datagen.flow_from_dataframe(dataframe=df, directory="images/", x_col="image", y_col="category", class_mode="categorical", target_size=(299,299), batch_size=32)

Found 6252 images belonging to 5 classes.


In [0]:
base_model =keras.applications.xception.Xception(input_shape=(299, 299, 3), weights='imagenet', include_top=False)

    # Top Model Block
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(5, activation='softmax')(x)

    # add your top layer block to your base model
model = Model(base_model.input, predictions)
print(model.summary())
for layer in model.layers[:20]:
    layer.trainable=False

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train['category']),
                                                 train['category'])
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=50,class_weight=class_weights)

In [0]:
model.save_weights("model.h5")
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [0]:
ls

In [0]:
test=pd.read_csv("test.csv")
sub=pd.read_csv("sample-submission.csv")

In [0]:
len(test)

2680

In [0]:
sub.head()

,image,category
0,1007700.jpg,1
1,1011369.jpg,1
2,1051155.jpg,1
3,1062001.jpg,1
4,1069397.jpg,1


In [27]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory="images/",
x_col="image",
y_col=None,
batch_size=2,
seed=42,
shuffle=False,
class_mode=None,
target_size=(299,299))

Found 2680 images.


In [28]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=loaded_model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

1340/1340 [==============================] - 30s 22ms/step


In [10]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Loaded model from disk


In [0]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"image":filenames,
                      "category":predictions})
results.to_csv("results.csv",index=False)

In [33]:
results.head()

,image,category
0,1007700.jpg,4
1,1011369.jpg,4
2,1051155.jpg,4
3,1062001.jpg,2
4,1069397.jpg,4


In [0]:
d1={'Cargo':1, 
'Military':2, 
'Carrier':3, 
'Cruise':4, 
'Tankers':5}
results['category'] .replace(d1, inplace=True)

In [0]:
results.to_csv("final.csv",index=False)

In [0]:
from google.colab import files
files.download('final.csv') 